In [1]:
sc

<SparkContext master=local[*] appName=pyspark-shell>

# Loading (remote) E-OBS observations and summer days (SU) calculation

Git repo: https://github.com/rmsryu/spark-climate-data.git
Data set: https://www.ecad.eu/download/ensembles/download.php

E-OBS data is loaded from the remote OpenDAP repository at KNMI. Please, note that when new versions become available the link is no longer valid since the URL is updated with the version number. Therefore, if the code below doesn't work please check the current E-OBS version and update the URL (https://www.ecad.eu/download/ensembles/download.php). 

# Study of daily precipitation from Station Aalsmeer Netherlands
Data source: https://climexp.knmi.nl/data/rrrr458.dat
Data provide by: Royal Netherlands Meteorological Institute (KNMI)

In [2]:
import numpy as np
import matplotlib.pyplot as plt
from pyspark import SparkFiles
from pyspark.sql.types import *
from pyspark.sql.functions import *



In [3]:
import warnings
warnings.filterwarnings("ignore")

In [4]:
hadoopUrl = 'hdfs://hadoop-vm.internal.cloudapp.net:9000'
data_files = f'{hadoopUrl}/precipitation/data/*/*.parquet'
# Obtain dataset
data = spark.read.parquet(data_files)
data = data.withColumn("precipitation", col("precipitation").cast("float"))
data = data.withColumn("date", col("date").cast("date"))
data.createOrReplaceTempView("precipitations")
data.cache()

# Read Station Info
schema_stations = StructType([
        StructField("station", StringType(), True),
        StructField("name", StringType(), True),
        StructField("latitude", FloatType(), True),
        StructField("longitude", FloatType(), True),
        StructField("website", StringType(), True),
        StructField("rawdata", StringType(), True),
        StructField("filename", StringType(), True),
        StructField("from", StringType(), True),
        StructField("to", StringType(), True)
    ])
station_files = 'hdfs://hadoop-vm.internal.cloudapp.net:9000/precipitation/data/stations/*.csv'
stations = spark.read.csv(station_files,schema=schema_stations)
stations.withColumn("from", col("from").cast("date"))
stations.withColumn("to", col("to").cast("date"))
stations.createOrReplaceTempView("stations")


In [5]:
data.summary().show()

+-------+------------------+-----------------+
|summary|           station|    precipitation|
+-------+------------------+-----------------+
|  count|          13120148|         13120148|
|   mean|483.02698064076714| 2.13989743152896|
| stddev|287.72368836944077|4.339730993138887|
|    min|               001|              0.0|
|    25%|             228.0|              0.0|
|    50%|             466.0|              0.1|
|    75%|             737.0|              2.4|
|    max|               983|            208.0|
+-------+------------------+-----------------+



In [6]:
stations.show()

+-------+--------------------+--------+---------+--------------------+--------------------+------------+----------+----------+
|station|                name|latitude|longitude|             website|             rawdata|    filename|      from|        to|
+-------+--------------------+--------+---------+--------------------+--------------------+------------+----------+----------+
|    458|            Aalsmeer|   52.25|     4.77|https://climexp.k...|https://climexp.k...|rrrr458.data|1927-01-02|2023-03-10|
|    040|   Aalsum bij Dokkum|  -999.9|   -999.9|https://climexp.k...|https://climexp.k...|rrrr040.data|1892-12-29|1925-09-30|
|    680|              Aalten|    51.9|     6.55|https://climexp.k...|https://climexp.k...|rrrr680.data|1904-08-02|2023-03-10|
|    711|          Aardenburg|  -999.9|   -999.9|https://climexp.k...|https://climexp.k...|rrrr711.data|1873-12-01|1887-06-30|
|    572|             Abcoude|   52.25|     4.97|https://climexp.k...|https://climexp.k...|rrrr572.data|1930-09

# Stations stats

In [7]:
spark.sql("SELECT station,\
              year(min(date)) min_year, \
              year(max(date)) - year(min(date)) num_years, \
              count(precipitation) count,\
              mean(precipitation) mean, \
              stddev(precipitation) stddev, \
              min(precipitation) min, \
              percentile(precipitation,0.25) 25pct, \
              percentile(precipitation,0.50) 50pct, \
              percentile(precipitation,0.75) 75pct, \
              max(precipitation) max \
            FROM precipitations GROUP BY station ORDER BY station").show()

+-------+--------+---------+-----+------------------+------------------+---+-----+-------------------+------------------+-----+
|station|min_year|num_years|count|              mean|            stddev|min|25pct|              50pct|             75pct|  max|
+-------+--------+---------+-----+------------------+------------------+---+-----+-------------------+------------------+-----+
|    001|    1940|       83|29556| 2.174519556030149| 4.370148656161713|0.0|  0.0|0.10000000149011612|               2.5| 76.6|
|    003|    1902|       51|18482|2.0838329184571815| 4.277456332945257|0.0|  0.0|0.10000000149011612| 2.299999952316284| 78.4|
|    004|    1904|        1|  419| 1.617899758558996|3.4800441747162214|0.0|  0.0|0.10000000149011612| 1.399999976158142| 26.9|
|    006|    1905|       38|13970|1.7968360776213679| 3.740703703355201|0.0|  0.0|0.10000000149011612| 1.899999976158142| 82.1|
|    007|    1906|       44|15101|1.9131845566100272|3.8756137261021313|0.0|  0.0|                0.0| 2

## Reliabality

To create a relevant and reliable dataset for average precipitation the following will be considered:

**Data selection**: Focus on the period with the most comprehensive and consistent data choosing a time frame when a higher number of weather stations are available.

**Handling missing data**: For stations with missing data or long gaps consider remove or imputate missing values.

**Temporal trends**: Analyze the dataset for any systematic temporal trends (e.g., increasing or decreasing precipitation over the years). 

**Weighting**: Contribution of each station based on factors like the length of the available data, the quality of the measurements, or the spatial coverage. Spatial coverage per station is 10Km

**Spatial coverage**: Stations spread homogenusly on the Netherland area. Some of stations do not include spatial information. 


In [8]:
from functools import reduce
from operator import add

def find_desired_year_stations(minimum_year = 1978, min_desired_data_points_per_year = 360):    
    # Get stations grouped by year and calculate desired number of records per year
    stations_years_complete  = spark.sql(f"SELECT station, year(date) year, count(precipitation) pre_year FROM precipitations where year(date) >= {minimum_year} GROUP BY station, year(date)")
    stations_years_complete = stations_years_complete.filter(col("pre_year") > min_desired_data_points_per_year)
    stations_years_complete = stations_years_complete.select("station","year").orderBy("station","year")

    # Group by 'station' and pivot on 'year'
    df_transposed = stations_years_complete.groupBy("station").pivot("year").avg("year")

    # Count the number of non null years per station
    year_columns = df_transposed.columns[1:]
    
    # expresion to filter year column not null
    count_years_expr = reduce(add, [when(col(year_col).isNotNull(), 1).otherwise(0) for year_col in year_columns])
    df_transposed = df_transposed.withColumn("total_years", count_years_expr)

    # print(f"Looking for stations with years {len(year_columns)}")
    stations_for_analysis = df_transposed.filter(col("total_years") == len(year_columns)).select("station","total_years")
    stations_filter = [item.station for item in stations_for_analysis.collect()]
    # print(f"A total of {len(stations_filter)} stations to be included")
    
    return stations_filter


In [9]:
starting_years = np.arange(1950, 1960)
# Iterate to find best year
year_list = []
result = []
best_num_of_stations = 0
for year in starting_years:
    temp = len(find_desired_year_stations(year, 360))
    result.append((year,temp))
    best_initial_year =  year if (temp > best_num_of_stations) else best_initial_year
    best_num_of_stations =  temp if (temp > best_num_of_stations) else best_num_of_stations

{"@version":1,"source_host":"rmsryu-vm","message":"Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.","thread_name":"Thread-4","@timestamp":"2023-04-06T18:12:23.602+0000","level":"WARN","logger_name":"org.apache.spark.sql.catalyst.util.package"}


In [10]:
result

[(1950, 132),
 (1951, 144),
 (1952, 150),
 (1953, 156),
 (1954, 174),
 (1955, 179),
 (1956, 188),
 (1957, 204),
 (1958, 212),
 (1959, 218)]

In [14]:
candiate_min_year = 1955
# Selecting year 1955 as baseline with 179 station complete stations datasets and minimum 360 observations
stations_filter = find_desired_year_stations(candiate_min_year, 360)

In [15]:
len(stations_filter)

179

In [16]:
data = data.filter(data.station.isin(stations_filter))
data.createOrReplaceTempView("final_ds")
df_final = spark.sql(f"SELECT date, avg(precipitation) precipitation, count(precipitation) w from final_ds where year(date) > {candiate_min_year} group by date")
df_final.summary().show()

+-------+-------------------+-------------------+
|summary|      precipitation|                  w|
+-------+-------------------+-------------------+
|  count|              24541|              24541|
|   mean| 2.2576964836099185| 178.99690314168126|
| stddev|  3.513736968249654|0.07864081706020871|
|    min|                0.0|                177|
|    25%|0.03240223528619585|                179|
|    50%| 0.6435754158476877|                179|
|    75%|  3.143575413183793|                179|
|    max|  45.87486042257128|                179|
+-------+-------------------+-------------------+



In [17]:
# Save to hadoop
df_final.write.mode("overwrite").parquet(f"{hadoopUrl}/precipitation/data/agg/from/{candiate_min_year}")

In [18]:
stations_filter

['829',
 '467',
 '666',
 '447',
 '740',
 '747',
 '442',
 '743',
 '462',
 '234',
 '672',
 '543',
 '073',
 '434',
 '974',
 '356',
 '087',
 '348',
 '541',
 '970',
 '584',
 '443',
 '910',
 '737',
 '670',
 '139',
 '559',
 '352',
 '160',
 '012',
 '570',
 '539',
 '843',
 '742',
 '973',
 '964',
 '749',
 '458',
 '565',
 '067',
 '573',
 '075',
 '327',
 '221',
 '081',
 '897',
 '163',
 '912',
 '453',
 '224',
 '019',
 '667',
 '449',
 '046',
 '140',
 '549',
 '905',
 '736',
 '835',
 '558',
 '147',
 '839',
 '455',
 '328',
 '228',
 '669',
 '011',
 '756',
 '341',
 '329',
 '967',
 '678',
 '229',
 '066',
 '076',
 '354',
 '578',
 '464',
 '439',
 '837',
 '840',
 '345',
 '238',
 '456',
 '901',
 '913',
 '914',
 '827',
 '156',
 '338',
 '830',
 '963',
 '754',
 '755',
 '965',
 '834',
 '450',
 '471',
 '968',
 '461',
 '741',
 '437',
 '751',
 '665',
 '673',
 '896',
 '832',
 '090',
 '016',
 '833',
 '148',
 '240',
 '239',
 '158',
 '838',
 '236',
 '564',
 '567',
 '444',
 '562',
 '336',
 '068',
 '907',
 '358',
 '899',
